# 1. What is Name in Python?
Name(identifier) is simply a name given to objects. Everything in Python is an **object**. Name is a way to access the underlying object, and a name could reger to any type of object. Actually, functions are objects too.

In [1]:
import os
print(os.getcwd())

C:\Users\dell\Desktop\jupter_CS100


In [2]:
t=2
print('id(2)=',id(t))
t='test_string'

def printHello(): # 没有输入参数依然需要"()"
    print("hello")

id(2)= 2005098688


# 2. What is Namespce in Python?
Namespace is a mapping from each name to its corresponding objects.This concept exits in ** Module, Class, def.** Different namespaces can c0-exist at a given time but are completely isolated. 

1. The namespace containing all the built-in names is created when we start the Python interpreter and exits as long we don't exit. (So this is the reasoning that built-in functions like id(), print() are always available to use from any part of the program.) Each module creates its own gloabl namespace.

2. The global namespace for a module is created when the module definition is read in; Normally, such global namespace also lasts until the interpreter quits.

3. The local namespace for a function is createtd when the funciton is called, and deleted when the function returns or raises an exception that is not handled within the function. 

Different namespaces are isolated.Hence, the same name that may exist in different modules don't collide.

# 3. What is Scope in Python?
Although there are various unique namespaces defined, we may not be able to access all of them from every part of the program. The concept of scope comes into play:

** Scope ** is the portion of the program from where a namespace can be accessed directly **without any prefix**.  中文来讲就是：每个scope都规定了在此scope中可以直接access的namespace，scope一定是namespace，但namespace不一定是scope。 

Although scopes are determined statically, they are used dynamically. At any time during execution, there are at least **three nested** scopes whose namespaces are directly accessible:
- the **innermost scope**, which is searched first, contains the local names
- the scopes of any **enclosing functions**, which are searched starting with the nearest enclosing scope, contains non-local, but also non-global names
- the next-to-last scope contains the current module’s **global** names
- the outermost scope (searched last) is the namespace containing **built-in** names
### Rule: L--E--G--B

In [3]:
x1="Global-Scope(G)"
def foo():
    x1="foo's Local Scope(L)"
    def bar():
        def baz():
            x1="baz's L-Scope(L)"
            print("baz calls: ",x1)
        baz()
#         baz(x1)
#         global x1
        print("bar calls: ",x1)
    print("foo calls: ",x1)
    bar()
    
print("global calls: ",x1)
foo()

global calls:  Global-Scope(G)
foo calls:  foo's Local Scope(L)
baz calls:  baz's L-Scope(L)
bar calls:  foo's Local Scope(L)


Python解释器在执行到bar()中的baz(x1)时，发现有个变量x1在当前scope(Local Namespace)中无法找到，但是会继续尝试把整块代码解释完。然而在下一行"global x1"中找到了，那么解释器便认为这里的x1属于Local Namespace的，既然对变量b的引用发生在了声明之前，自然会抛出错误。

**Nonlocal statement** name x to previously bound variables in the nearest **Enclosing Fucntion**, but **not** Global Scope!

In [4]:
x2="global"
def foo():
    x2="Enclosing Fucntion - 1"
    def bar():
        x2="Enclosing Fucntion - 2"
        def baz():
            nonlocal x2
            print(x2)
        baz()
    bar()
foo()

Enclosing Fucntion - 2


对于python，def/class/lambda表达式均会产生新的namespace,其他如：if/elif/else try/except for/while并不会产生namespace，自然也不存在作用域一说了。所以对于后者来说，即便在它们内部定义的变量，在外部也可以访问。

Now look at following usages of class:

In [5]:
class A(object):
    a1=4 # protected class attribute, which cannot be imported using 'from module import *'
    print(a1)

4


In [6]:
class A(object):
    a2=4
    def printA2(self):
        print(a2)
        
A().printA2()

NameError: name 'a2' is not defined

In [7]:
class A(object):
    a3=4
    def printA1(self):
        print(self.a3)
a=A().printA1()

4


In [8]:
class B(object):
    a1=4    
    b1=[_a1+i for i in range(10)]
    print(_b1)    

NameError: name '_a1' is not defined

In [9]:
a2=40
class B(object):
    a2=4
    b2=[a2+i for i in range(10)]
    print(b2)

[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


原因是： 对于python3，列表推导式也会引入一个新的namespace，此namespace将作为此列表推导式的local scope. 在定义一个class时也会产生一个新的作用域作为此class的local scope：The inside of a class body is also a new local variable scope. This new scope can never be gobal scope! Varible which are defined in the class body(but outside any class method) are called class atrributes. They can be referenced by their bare names within the same scope, but they can also be accessed from outside this scope, if we use the attribute access operator(.) on a class or an instance.当列表推导式遵循L-->E-->G-->B规则寻找时，(Note：这里的LEGB均是scope)，只有L，G，B这三个scope可以被列表推导式access到。因为E的定义是：the scopes of any ** enclosing functions **,而**B是class不是function**，所以对于列表推导式而言是没有E这个scope的，因此列表推导式将再次向上一层scope寻找并获取到G(Global scope)中的a3=40.

注：这里说的无法获取class里面a3指的是无法直接获取(barely without attribute access operator '.'),当我们传入一个instance给function，自然也是可以获取到相应的attribute的。

In [10]:
[m for m in range(10)]
print(m)

NameError: name 'm' is not defined

Solution: 间接传参

In [11]:
class B(object):
    a3=4 
    def getB3(self):
        b3=[self.a3+i for i in range(10)]
        print(b3)

B().getB3()

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [12]:
class B(object):
    a4=4
    b4=(lambda a: [a + i for i in range(10)])
    print(b4(a4))

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


Python除了def/class/lambda以及列表推导式外，其他如：if/elif/else try/except for/while并不会引入新的作用域。对于后者来说，即便在它们内部定义的变量，在外部也可以访问。

In [13]:
if True:
    # this is still a global variable
    x3='python_if_scope'
    print(x3)

print(x3)

python_if_scope
python_if_scope


**注意：**for后面跟着的变量(target list)在循环结束后不会被删除：for-loop variables “leaking” into the global namespace

In [14]:
for i in range(10): 
    print(i)
    
print("continue print outof ForLoop: ", i)

0
1
2
3
4
5
6
7
8
9
continue print outof ForLoop:  9
